# Analyze the effect of antipsychotics on gene expression

In [ ]:
library(tidyverse)
library(ggpubr)

## Functions

In [ ]:
save_img <- function(image, fn, w, h){
    for(ext in c(".svg", ".pdf", ".png")){
        ggsave(file=paste0(fn, ext), plot=image, width=w, height=h)
    }
}

get_pheno <- function(){
    phenotypes <- data.table::fread(paste0("/ceph/projects/v4_phase3_paper/inputs/",
                                           "phenotypes/_m/merged_phenotypes.csv")) %>% 
        mutate_if(is.character, as.factor)
    return(phenotypes)
}

memPHENO <- memoise::memoise(get_pheno)

In [ ]:
pca_select <- function(norm_df){
    ### Dimensional reduction (PCA)
    log2cpm = norm_df %>% column_to_rownames("V1") %>% t
    pca_df = prcomp(log2cpm, center=TRUE)$x
    dt = pca_df %>% as.data.frame %>% rownames_to_column() %>% 
        pivot_longer(-rowname, names_to="PC", values_to="PC_values")
    return(dt)
}

memPCA <- memoise::memoise(pca_select)

## Normalized expression

In [ ]:
memPHENO() %>% filter(Dx %in% c("CTL", "SZ"), Age > 13, 
                      Region == "Caudate") %>% 
    group_by(Dx, antipsychotics) %>% summarize(N=n())

In [ ]:
norm_df <- data.table::fread("../../main_plots/_m/caudate/normalized_expression.tsv")
norm_df[1:2, 1:5]

In [ ]:
dt = memPCA(norm_df) %>% mutate_if(is.character, as.factor)
dt %>% dim
dt %>% head(2)

In [ ]:
set.seed(20210723)
bxp = dt %>% filter(PC %in% c("PC1", "PC2", "PC3", "PC4", "PC5")) %>%
    inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na() %>%
    ggboxplot(x='antipsychotics', y="PC_values", fill='antipsychotics', add='jitter', 
              palette="npg", facet.by="PC", legend='bottom', add.params=list(alpha=0.75),
              panel.labs.font=list(face='bold'), ncol=5, ylim=c(-130, 130), 
              ggtheme=theme_pubr(base_size=20)) + 
        stat_compare_means(comparisons=list(c("TRUE", "FALSE")), aes(label=..p.signif..), 
                           method="wilcox.test") +
        font("title", color="black", face="bold") + font("xy.title", face="bold") + 
        font("legend.title", face="bold")
save_img(bxp, paste0("boxplot_antipsychotics_normalized"), w=12, h=4)
print(bxp)

In [ ]:
set.seed(20210723)
bxp = dt %>% filter(PC %in% c("PC1", "PC2", "PC3", "PC4", "PC5")) %>%
    inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na() %>%
    filter(Dx == "SZ") %>%
    ggboxplot(x='antipsychotics', y="PC_values", fill='antipsychotics', add='jitter', 
              palette="npg", facet.by="PC", legend='bottom', add.params=list(alpha=0.75),
              panel.labs.font=list(face='bold'), ncol=5, ylim=c(-130, 130), 
              ggtheme=theme_pubr(base_size=20)) + 
    stat_compare_means(comparisons=list(c("TRUE", "FALSE")), aes(label=..p.signif..), 
                       method="wilcox.test") +
    font("title", color="black", face="bold") + font("xy.title", face="bold") + 
    font("legend.title", face="bold")
save_img(bxp, paste0("boxplot_antipsychotics_normalized_szOnly"), w=12, h=4)
print(bxp)

In [ ]:
set.seed(20210723)
bxp = dt %>% filter(PC %in% c("PC1", "PC2", "PC3", "PC4", "PC5")) %>%
    inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na() %>%
    ggboxplot(x='Dx', y="PC_values", fill='Dx', add='jitter', 
              palette="npg", facet.by="PC", legend='bottom', add.params=list(alpha=0.75),
              panel.labs.font=list(face='bold'), ncol=5, ylim=c(-130, 130), 
              ggtheme=theme_pubr(base_size=20)) + 
    stat_compare_means(comparisons=list(c("CTL", "SZ")), aes(label=..p.signif..), 
                       method="wilcox.test") +
    font("title", color="black", face="bold") + font("xy.title", face="bold") + 
    font("legend.title", face="bold")
save_img(bxp, paste0("boxplot_dx_normalized"), w=12, h=4)
print(bxp)

### Statistical test for all significant PCs for antipsychotics

In [ ]:
antipsychotics = dt %>% inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na()
pcs = c()
pvals = c()
for(pc in unique(antipsychotics$PC)){
    flush.console()
    dx = antipsychotics %>% filter(PC == pc)
    res = wilcox.test(filter(dx, antipsychotics=="TRUE")$PC_values,
                      filter(dx, antipsychotics=="FALSE")$PC_values)
    if(res$p.value < 0.05){
        pcs = c(pcs, pc)
        pvals = c(pvals, res$p.value)
    }
}

In [ ]:
pcs

## Residualized

In [ ]:
res_df <- data.table::fread("../../main_plots/_m/caudate/residualized_expression.tsv")
res_df[1:2, 1:5]

In [ ]:
dt = memPCA(res_df) %>% mutate_if(is.character, as.factor)
dt %>% dim
dt %>% head(2)

In [ ]:
set.seed(20210723)
bxp = dt %>% filter(PC %in% c("PC1", "PC2", "PC3", "PC4", "PC5")) %>%
    inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na() %>%
    ggboxplot(x='antipsychotics', y="PC_values", fill='antipsychotics', add='jitter', 
              palette="npg", facet.by="PC", legend='bottom', add.params=list(alpha=0.75),
              panel.labs.font=list(face='bold'), ncol=5, ylim=c(-130, 130), 
              ggtheme=theme_pubr(base_size=20)) + 
    stat_compare_means(comparisons=list(c("TRUE", "FALSE")), aes(label=..p.signif..), 
                       method="wilcox.test") +
    font("title", color="black", face="bold") + font("xy.title", face="bold") + 
    font("legend.title", face="bold")
save_img(bxp, paste0("boxplot_antipsychotics_residualized"), w=12, h=4)
print(bxp)

In [ ]:
set.seed(20210723)
bxp = dt %>% filter(PC %in% c("PC1", "PC2", "PC3", "PC4", "PC5")) %>%
    inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na() %>%
    filter(Dx == "SZ") %>%
    ggboxplot(x='antipsychotics', y="PC_values", fill='antipsychotics', add='jitter', 
              palette="npg", facet.by="PC", legend='bottom', add.params=list(alpha=0.75),
              panel.labs.font=list(face='bold'), ncol=5, ylim=c(-130, 130), 
              ggtheme=theme_pubr(base_size=20)) + 
    stat_compare_means(comparisons=list(c("TRUE", "FALSE")), aes(label=..p.signif..), 
                       method="wilcox.test") +
    font("title", color="black", face="bold") + font("xy.title", face="bold") + 
    font("legend.title", face="bold")
save_img(bxp, paste0("boxplot_antipsychotics_residualized_szOnly"), w=12, h=4)
print(bxp)

In [ ]:
set.seed(20210723)
bxp = dt %>% filter(PC %in% c("PC1", "PC2", "PC3", "PC4", "PC5")) %>%
    inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na() %>%
    ggboxplot(x='Dx', y="PC_values", fill='Dx', add='jitter', 
              palette="npg", facet.by="PC", legend='bottom', add.params=list(alpha=0.75),
              panel.labs.font=list(face='bold'), ncol=5, ylim=c(-130, 130), 
              ggtheme=theme_pubr(base_size=20)) + 
    stat_compare_means(comparisons=list(c("CTL", "SZ")), aes(label=..p.signif..), 
                       method="wilcox.test") +
    font("title", color="black", face="bold") + font("xy.title", face="bold") + 
    font("legend.title", face="bold")
save_img(bxp, paste0("boxplot_dx_residualized"), w=12, h=4)
print(bxp)

### Statistical test for all significant PCs for antipsychotics

In [ ]:
antipsychotics = dt %>% inner_join(memPHENO(), by=c("rowname"="RNum")) %>% drop_na()
pcs2 = c()
pvals2 = c()
for(pc in unique(antipsychotics$PC)){
    flush.console()
    dx = antipsychotics %>% filter(PC == pc)
    res = wilcox.test(filter(dx, antipsychotics=="TRUE")$PC_values,
                      filter(dx, antipsychotics=="FALSE")$PC_values)
    if(res$p.value < 0.05){
        pcs2 = c(pcs2, pc)
        pvals2 = c(pvals2, res$p.value)
    }
}

pcs2

## Reproducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()